In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as stats
import statsmodels.api as sm

In [1]:
from utils.funciones import *

ModuleNotFoundError: No module named 'utils'

In [3]:
import warnings
warnings.filterwarnings('ignore')

## Consideraciones generales

¡Ojo! Los animes están separados por temporadas. ¿Conviene que haya solo una fila por anime, sumando datos cuando sea necesario (como x ejemplo núm de personas que la puntuaron) y haciendo media de datos cuando sea necesario (la puntuación, x ejemplo)?

### Data Cleaning
- Se elimina la columna "ID", "Synonyms", "Japanese", "Synopsis" , "End_Aired", "Producers", Licensors", "Popularity", "Members", "Favourite" y "Broadcast" por no aportar datos relevantes o por exceso de "Unknowns"
- Se plantea eliminar la columna "Title" y renombrar la columna "English" por "Title". Sin embargo, aunque eso lo haría más comprensible para el público general, se comprueba que la columna "English" tiene más de 11K "Unknown". Se decide dejar ambas columnas
- Se eliminan las filas donde "Title" == "Unknown" porque no aporta valor la fila de un anime sin título
- Se eliminan las filas donde "Start_Aired" == "Unknown" porque no aporta valor el desconocimiento de la fecha en la que se estrena.
- Se eliminan las filas donde "Demographics" == "Unknown" porque para el posterior análisis estadístico es imprescindible que no haya Unknowns.
- En las columnas "Demographics", "Themes" y "Genres" se reemplazan los valores que tienen más de una palabra y se añaden a la categoría de la primera palabra. Por ejemplo, en "Demographics", "Kids, Shounen" se reemplaza por "Kids". Al ser una categoría ya existente, se suman los valores


### Missing values
    1. Tras hacer dos pruebas donde, por un lado, se reemplazan los NaN por la media de los valores en las columnas "Score" y "Scored_User"   y,  por otro lado, se eliminan las filas de esas columnas cuando hay un NaN, se decide mantener las filas y rellenarlas con el valor medio de esas columnas
    2. Se decide hacer lo mismo con las columnas "Episodes" y "Duration_Minutes", tras comprobar que los NaN aparecen tanto en series, como en pelis.
    3. En "Ranked" no tiene sentido rellenar los NaN con la media, porque es un ranking del 1 al **. Tras verificar la columna Ranked, se decide eliminar las filas donde "Ranked" == NaN. Se comprueba además que eliminando los Unknown de "Ranked" desaparecen los "Status" == "Not yet aired", dejando datos que tienen más valor para nuestras hipótesis

## Posibles Hipótesis

- Los 10 animes con mejor ranking son Shounen (demográfica)
- La puntuación está relacionada de forma inversa con la duración (a mayor duración, menor puntuación)
- Los animes cuya emisión empezó antes de los 2010 tienen peor puntuación que los animes que empezaron a emitirse después del 01/01/2010
- Los animes de género comedia tienen más posibilidades de tener una puntuación superior a 4/5
- La puntuación de las películas es inferior a las de las series
- Los animes de aventura tienen más usuarios que puntúan en comparación a comedia, pero la puntuación media es menor
-~~Los animes que se emiten en fin de semana (viernes, sábado, domingo) tienen mejor puntuación que los que se emiten entre semana~~ DESCARTADO. La columna Broadcast tiene demasiados Unknowns
- Los animes basados en manga tienen una puntuación media superior a los animes originales - histograma (pero se puede hacer con estadística)/ boxplot

In [5]:
anime = pd.read_csv('src\data\raw\Anime.csv', sep=",")
anime.head(3)

FileNotFoundError: [Errno 2] No such file or directory: 'src\\data\raw\\Anime.csv'

In [ ]:
anime.tail(3)

,ID,Title,Synonyms,Japanese,English,Synopsis,Type,Episodes,Status,Start_Aired,...,Themes,Demographics,Duration_Minutes,Rating,Score,Scored_Users,Ranked,Popularity,Members,Favorites
21457,45059,Yongzhe Chuang Mo Cheng,Unknown,勇者闯魔城,Brave City,Unknown,Movie,1.0,Finished Airing,"Mar 16, 2018",...,Unknown,Unknown,80.0,G - All Ages,NaN,NaN,164032.0,21489,6,0
21458,45063,Yu Shen Chuan: Wangzhe Shaonian,Unknown,禹神传之王者少年,The Juvenile of King Yu,Unknown,Movie,1.0,Finished Airing,"Aug 15, 2018",...,Unknown,Unknown,90.0,G - All Ages,NaN,NaN,164462.0,21490,6,0
21459,45061,Konglong Wang,Unknown,恐龙王,Dino King,Unknown,Movie,1.0,Finished Airing,"Nov 10, 2018",...,Historical,Kids,95.0,G - All Ages,NaN,NaN,193202.0,21490,6,0


In [ ]:
anime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21460 entries, 0 to 21459
Data columns (total 28 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ID                21460 non-null  int64  
 1   Title             21460 non-null  object 
 2   Synonyms          21460 non-null  object 
 3   Japanese          21460 non-null  object 
 4   English           21460 non-null  object 
 5   Synopsis          21460 non-null  object 
 6   Type              21460 non-null  object 
 7   Episodes          20913 non-null  float64
 8   Status            21460 non-null  object 
 9   Start_Aired       21460 non-null  object 
 10  End_Aired         21460 non-null  object 
 11  Premiered         21460 non-null  object 
 12  Broadcast         21460 non-null  object 
 13  Producers         21460 non-null  object 
 14  Licensors         21460 non-null  object 
 15  Studios           21460 non-null  object 
 16  Source            21460 non-null  object

In [ ]:
anime.describe()

,ID,Episodes,Duration_Minutes,Score,Scored_Users,Ranked,Popularity,Members,Favorites
count,21460.000000,20913.000000,20861.000000,14562.000000,14562.000000,19536.000000,21460.000000,2.146000e+04,21460.000000
mean,26458.575582,13.272892,23.934135,6.421724,26507.173809,97795.212019,10743.103681,3.926596e+04,462.214865
std,17100.477121,48.792326,24.676538,0.910469,84989.501599,56485.384273,6204.685188,1.566359e+05,4411.931536
min,1.000000,1.000000,1.000000,1.841000,101.000000,12.000000,1.000000,6.000000e+00,0.000000
25%,8454.750000,1.000000,5.000000,5.791000,418.000000,48879.500000,5369.750000,2.910000e+02,0.000000
50%,31004.500000,2.000000,22.000000,6.431000,1876.500000,97777.000000,10741.500000,1.582500e+03,2.000000
75%,40495.250000,12.000000,26.000000,7.081000,11406.500000,146714.500000,16119.250000,1.158550e+04,23.000000
max,53094.000000,3057.000000,168.000000,9.131000,997243.000000,195632.000000,21490.000000,3.524109e+06,207772.000000


In [ ]:
anime.columns

Index(['ID', 'Title', 'Synonyms', 'Japanese', 'English', 'Synopsis', 'Type',
       'Episodes', 'Status', 'Start_Aired', 'End_Aired', 'Premiered',
       'Broadcast', 'Producers', 'Licensors', 'Studios', 'Source', 'Genres',
       'Themes', 'Demographics', 'Duration_Minutes', 'Rating', 'Score',
       'Scored_Users', 'Ranked', 'Popularity', 'Members', 'Favorites'],
      dtype='object')

In [ ]:
anime.isnull().sum()


ID                     0
Title                  0
Synonyms               0
Japanese               0
English                0
Synopsis               0
Type                   0
Episodes             547
Status                 0
Start_Aired            0
End_Aired              0
Premiered              0
Broadcast              0
Producers              0
Licensors              0
Studios                0
Source                 0
Genres                 0
Themes                 0
Demographics           0
Duration_Minutes     599
Rating                 0
Score               6898
Scored_Users        6898
Ranked              1924
Popularity             0
Members                0
Favorites              0
dtype: int64

In [ ]:
#eliminamos las columnas que no necesitamos. Para no tener que crear nuevo dataframe, usamos inplace=True
anime.drop(["ID", "Synonyms", "Japanese", "Synopsis", "End_Aired", "Broadcast", "Producers", "Licensors", "Premiered"], axis= 1, inplace=True)

NameError: name 'anime' is not defined

In [ ]:
anime[["Status"]].value_counts()

Status          
Finished Airing     20693
Not yet aired         442
Currently Airing      325
dtype: int64

In [ ]:
#Missing values
#Escenario 1: Eliminar todas las filas que no tengan score. De poco nos sirven para hacer hipótesis luego, pero representan el 32% de las filas

# anime_delete_nan = anime.dropna(subset=["Score", "Scored_Users"], axis=0)

# #Escenario 2: Fill value con la media de Score y Score_users

# anime_fill_mean = anime.fillna({"Score": anime["Score"].mean(), "Scored_Users": anime["Scored_Users"].mean()})

In [ ]:
# anime.describe()

In [ ]:
# anime_fill_mean.describe()

In [ ]:
# anime_delete_nan.describe()

In [ ]:
#Se decide fill value con la media de Score y Score_users
anime.fillna({"Score": anime["Score"].mean(), "Scored_Users": anime["Scored_Users"].mean()}, inplace=True)

In [ ]:
#Verificamos que los episodios que tienen un NaN son de diferentes tipos ( hay animes, pelis, etc), por lo que se pueden
#rellenar con la media
anime.loc[anime["Episodes"].isnull()]. head()

,Title,English,Type,Episodes,Status,Start_Aired,Premiered,Studios,Source,Genres,Themes,Demographics,Duration_Minutes,Rating,Score,Scored_Users,Ranked,Popularity,Members,Favorites
24,One Piece,One Piece,TV,NaN,Currently Airing,"Oct 20, 1999",Fall 1999,Toei Animation,Manga,"Action, Adventure, Fantasy",Unknown,Shounen,24.0,PG-13 - Teens 13 or older,8.671000,121979.000000,582.0,25,1967217,177620
194,Boruto: Naruto Next Generations,Boruto Naruto Next Generations,TV,NaN,Currently Airing,"Apr 5, 2017",Spring 2017,Pierrot,Manga,"Action, Adventure, Fantasy",Martial Arts,Shounen,23.0,PG-13 - Teens 13 or older,5.781000,386342.000000,100332.0,195,773091,6687
362,Chainsaw Man,Chainsaw Man,TV,NaN,Not yet aired,"Oct 12, 2022",Fall 2022,MAPPA,Manga,"Action, Adventure",Gore,Shounen,NaN,None,6.421724,26507.173809,NaN,364,509913,14181
626,Detective Conan,Case Closed,TV,NaN,Currently Airing,"Jan 8, 1996",Winter 1996,TMS Entertainment,Manga,"Adventure, Comedy, Mystery",Detective,Shounen,25.0,PG-13 - Teens 13 or older,8.161000,147429.000000,3662.0,627,316645,13059
813,Mob Psycho 100 III,Mob Psycho 100 III,TV,NaN,Not yet aired,"Oct 6, 2022",Fall 2022,Bones,Web manga,"Action, Comedy, Supernatural",Super Power,Unknown,NaN,PG-13 - Teens 13 or older,6.421724,26507.173809,NaN,814,250031,2785


In [ ]:
#Verificamos que la duración que tiene un NaN son de diferentes tipos, por lo que se pueden rellenar con la media
anime.loc[anime["Duration_Minutes"].isnull()].head()

,Title,English,Type,Episodes,Status,Start_Aired,Premiered,Studios,Source,Genres,Themes,Demographics,Duration_Minutes,Rating,Score,Scored_Users,Ranked,Popularity,Members,Favorites
362,Chainsaw Man,Chainsaw Man,TV,NaN,Not yet aired,"Oct 12, 2022",Fall 2022,MAPPA,Manga,"Action, Adventure",Gore,Shounen,NaN,None,6.421724,26507.173809,NaN,364,509913,14181
813,Mob Psycho 100 III,Mob Psycho 100 III,TV,NaN,Not yet aired,"Oct 6, 2022",Fall 2022,Bones,Web manga,"Action, Comedy, Supernatural",Super Power,Unknown,NaN,PG-13 - Teens 13 or older,6.421724,26507.173809,NaN,814,250031,2785
833,Tate no Yuusha no Nariagari Season 3,The Rising of the Shield Hero Season 3,TV,NaN,Not yet aired,Unknown,Unknown,Kinema Citrus,Light novel,"Action, Adventure, Drama, Fantasy",Isekai,Unknown,NaN,PG-13 - Teens 13 or older,6.421724,26507.173809,NaN,834,244491,2855
890,Spy x Family Part 2,Unknown,TV,13.0,Not yet aired,"Oct 1, 2022",Fall 2022,"Wit Studio, CloverWorks",Manga,"Action, Comedy",Childcare,Shounen,NaN,PG-13 - Teens 13 or older,6.421724,26507.173809,NaN,891,229447,1628
915,Boku no Hero Academia 6th Season,My Hero Academia Season 6,TV,NaN,Not yet aired,"Oct 1, 2022",Fall 2022,Bones,Manga,Action,"School, Super Power",Shounen,NaN,PG-13 - Teens 13 or older,6.421724,26507.173809,NaN,916,223328,1877


In [ ]:
#Quiero comprobar si rellenar los NaN de "Episode" con la media es realista, o va a afectar a muchas películas.
#en Type, cuántos tipos hay y cuáles son Unknown?
# anime["Type"].value_counts()
# anime[anime["Type"] == "Unknown"]
# Descomentando el código anterior, vemos que solo hay 45 unknowns. Buscando unos cuantos en google vemos que hay series en su mayoría.
#Resumiendo: Es buena idea rellenar los Nan de "Episode" con la media de la columna

In [ ]:
anime.fillna({"Episodes": anime["Episodes"].mean(), "Duration_Minutes": anime["Duration_Minutes"].mean()}, inplace=True)

In [ ]:
#Verificamos la columna "Ranked". Justo los que no están en el ranking, tienen muchísimos Unknowns en otras columnas. Mejor eliminarlos.
# ranked_nan = anime[anime['Ranked'].apply(lambda x: np.isnan(x))]
# ranked_nan

In [ ]:
#Eliminamos los Nan de "Ranked"
anime.dropna(subset=["Ranked"], axis=0, inplace=True)
anime.isnull().sum()

Title               0
English             0
Type                0
Episodes            0
Status              0
Start_Aired         0
Premiered           0
Studios             0
Source              0
Genres              0
Themes              0
Demographics        0
Duration_Minutes    0
Rating              0
Score               0
Scored_Users        0
Ranked              0
Popularity          0
Members             0
Favorites           0
dtype: int64

In [ ]:
anime[["Demographics"]].value_counts()
#Hay un montón de Unknowns!!!

Demographics 
Unknown          11617
Kids              4219
Shounen           1972
Seinen             874
Shoujo             662
Josei               92
Kids, Shounen       66
Kids, Shoujo        32
Josei, Shoujo        1
Kids, Seinen         1
dtype: int64

In [ ]:
anime = anime.drop(index=anime.loc[anime['Demographics'] == 'Unknown'].index)

In [ ]:
#Vamos a intentar dejar solo Kids, Shounen, Seinen, Shoujo y Josei. Las que tienen varias parabras, las unimos a la primera 

anime["Demographics"] = anime["Demographics"].str.replace('^Kids.*', 'Kids').str.replace('^Josei.*', 'Josei')
anime["Demographics"].value_counts()

Kids       4318
Shounen    1972
Seinen      874
Shoujo      662
Josei        93
Name: Demographics, dtype: int64

In [ ]:
anime[["Themes"]].value_counts()

Themes                                
Unknown                                   3701
Music                                      603
School                                     314
Historical                                 254
Anthropomorphic                            210
                                          ... 
Adult Cast, Psychological, Team Sports       1
Historical, Military, Mythology              1
Historical, Mythology, School                1
Historical, Mythology, Space                 1
Historical, Racing                           1
Length: 442, dtype: int64

In [ ]:
#Vamos a intentar dejar solo Temas con 1 palabra. Las que tienen varias parabras, las unimos a la primera. No sé hacerlo todo junto, así que he tenido que hacerlo uno por uno. 
#Explicacion del código: con .str.replace pedimos que reemplace una palabra por otra.
#^ indica el inicio de la línea o cadena de texto. Por lo tanto, ^Martial Arts significa "todas las líneas o cadenas de texto que comienzan con 'Martial Arts'".
# . significa cualquier caracter excepto el salto de línea. Por lo tanto, ^Martial Arts.* significa "todas las líneas o cadenas de texto que comienzan con 'Martial Arts'
# seguido de cualquier cosa".
# * significa cero o más repeticiones del elemento anterior. Por lo tanto, ^Martial Arts.* significa "todas las líneas o cadenas de texto que comienzan con 'Martial Arts'
# seguido de cualquier cosa, incluyendo la ausencia de cualquier cosa". 

anime['Themes'] = anime['Themes'].str.replace('^Martial Arts.*', 'Martial Arts').str.replace('^Historical.*', 'Historical').str.replace('^Harem.*', 'Harem').str.replace('^Mecha.*',
                     'Mecha').str.replace('^Crossdressing.*','Crossdressing').str.replace('^Educational.*', 'Educational').str.replace('^Adult Cast.*', 'Adult Cast').str.replace('^Gag Humor.*',
                     'Gag Humor').str.replace('^Gore.*', 'Gore').str.replace('^CGDCT.*', 'CGDCT').str.replace('^Detective.*', 'Detective').str.replace('^Mythology.*', 
                    'Mythology').str.replace('^Anthropomorphic.*', 'Anthropomorphic').str.replace('^Magical Sex Shift.*', 'Magical Sex Shift').str.replace('^Iyashikei.*', 'Iyashikei').str.replace('^Idols.*',
                     'Idols').str.replace('^Space.*', 'Space').str.replace('^Music.*', 'Music').str.replace('^Mahou Shoujo.*', 'Mahou Shoujo').str.replace('^School.*', 'School').str.replace('^Combat Sports.*',
                     'Combat Sports').str.replace('^Isekai.*', 'Isekai').str.replace('^Reverse Harem.*', 'Reverse Harem').str.replace('^Childcare.*', 'Childcare').str.replace('^Strategy Game.*',
                    'Strategy Game').str.replace('^Psychological.*', 'Psychological').str.replace('^Racing.*', 'Racing').str.replace('^Love Polygon.*', 'Love Polygon').str.replace('^Delinquents.*',
                     'Delinquents').str.replace('^Parody.*', 'Parody').str.replace('^Military.*', 'Military').str.replace('^Organized Crime.*', 'Organized Crime').str.replace('^Otaku Culture.*',
                    'Otaku Culture').str.replace('^High Stakes Game.*', 'High Stakes Game').str.replace('^Survival.*', 'Survival').str.replace('^Pets.*', 'Pets').str.replace('^Reincarnation.*',
                     'Reincarnation').str.replace('^Samurai.*', 'Samurai').str.replace('^Performing Arts.*', 'Performing Arts').str.replace('^Showbiz.*', 'Showbiz').str.replace('^Super Power.*',
                     'Super Power').str.replace('^Romantic Subtext.*', 'Romantic Subtext').str.replace('^Medical.*', 'Medical')
anime["Themes"].value_counts()

Unknown              3701
Music                 656
Historical            428
School                413
Mecha                 302
Anthropomorphic       243
Educational           181
Adult Cast            152
Super Power           136
Mythology             125
Martial Arts          118
Mahou Shoujo          116
Detective              95
Harem                  94
Gag Humor              92
Team Sports            90
Racing                 89
Strategy Game          86
Parody                 69
Idols                  58
Combat Sports          58
Iyashikei              57
Gore                   50
Space                  45
CGDCT                  42
Psychological          41
Military               40
Delinquents            31
Isekai                 30
Childcare              28
Vampire                28
Love Polygon           26
Pets                   22
Reverse Harem          20
Organized Crime        17
Crossdressing          17
Otaku Culture          14
Magical Sex Shift      14
Workplace   

In [ ]:
anime[["Genres"]].value_counts()

Genres                                              
Unknown                                                 1975
Comedy                                                   566
Fantasy                                                  457
Adventure, Fantasy                                       215
Drama                                                    204
                                                        ... 
Action, Drama, Horror, Romance, Supernatural               1
Comedy, Fantasy, Gourmet                                   1
Comedy, Fantasy, Horror, Supernatural                      1
Action, Drama, Horror, Romance, Sci-Fi, Supernatural       1
Adventure, Fantasy, Horror, Sci-Fi, Supernatural           1
Length: 475, dtype: int64

In [ ]:
def genresclean(animecol):
    animecol = animecol.str.replace('^Adventure.*', 'Adventure').str.replace('^Action.*', 'Action').str.replace('^Comedy.*', 'Comedy').str.replace('^Fantasy.*',
                'Fantasy').str.replace('^Mystery.*', 'Mystery').str.replace('^Drama.*', 'Drama').str.replace('^Avant Garde.*', 'Avant Garde').str.replace('^Boys Love.*',
                'Boys Love').str.replace('^Girls Love.*', 'Girls Love').str.replace('^Sports.*', 'Sports').str.replace('^Horror.*', 'Horror').str.replace('^Romance.*',
                'Romance').str.replace('^Award Winning.*', 'Award Winning').str.replace('^Gourmet.*', 'Gourmet').str.replace('^Sci-Fi.*', 'Sci-Fi').str.replace('^Slice of Life.*',
                'Slice of Life').str.replace('^Supernatural.*', 'Supernatural')
    return animecol


In [ ]:
anime["Genres"] = anime["Genres"].apply(genresclean)

In [ ]:
anime["Genres"]= anime["Genres"].str.replace('^Adventure.*', 'Adventure').str.replace('^Action.*', 'Action').str.replace('^Comedy.*', 'Comedy').str.replace('^Fantasy.*',
                'Fantasy').str.replace('^Mystery.*', 'Mystery').str.replace('^Drama.*', 'Drama').str.replace('^Avant Garde.*', 'Avant Garde').str.replace('^Boys Love.*',
                'Boys Love').str.replace('^Girls Love.*', 'Girls Love').str.replace('^Sports.*', 'Sports').str.replace('^Horror.*', 'Horror').str.replace('^Romance.*',
                'Romance').str.replace('^Award Winning.*', 'Award Winning').str.replace('^Gourmet.*', 'Gourmet').str.replace('^Sci-Fi.*', 'Sci-Fi').str.replace('^Slice of Life.*',
                'Slice of Life').str.replace('^Supernatural.*', 'Supernatural')
anime["Genres"].value_counts()

Unknown          1975
Comedy           1646
Action           1561
Adventure        1055
Fantasy           536
Drama             461
Sports            175
Slice of Life     147
Sci-Fi            117
Romance            94
Mystery            46
Supernatural       34
Horror             23
Gourmet            22
Boys Love          11
Avant Garde         6
Suspense            4
Ecchi               3
Girls Love          3
Name: Genres, dtype: int64

In [ ]:
anime["Start_Aired"].value_counts()


Unknown         126
2020             56
2010             50
2011             43
2005             41
               ... 
Jun 28, 2009      1
Jul 29, 2019      1
Jun 22, 1985      1
Jul 11, 2009      1
Nov 10, 2018      1
Name: Start_Aired, Length: 4348, dtype: int64

In [ ]:
mask=(anime["Start_Aired"] == 'Unknown')
anime["Start_Aired"].loc[mask]

13116    Unknown
13151    Unknown
13762    Unknown
14133    Unknown
14782    Unknown
          ...   
21426    Unknown
21446    Unknown
21447    Unknown
21449    Unknown
21451    Unknown
Name: Start_Aired, Length: 126, dtype: object

In [ ]:
mask=(anime["Start_Aired"] == "1900")
anime["Start_Aired"].loc[mask]

Series([], Name: Start_Aired, dtype: object)

In [ ]:
for i in anime["Start_Aired"]:
    if i == 'Unknown':
        anime["Start_Aired"][anime["Start_Aired"] == 'Unknown'] = 1900

In [ ]:
mask=(anime["Start_Aired"] == 'Apr-03')
anime["Start_Aired"].loc[mask]

5129    Apr-03
Name: Start_Aired, dtype: object

In [ ]:
anime["Start_Aired"].value_counts()

1900            126
2020             56
2010             50
2011             43
2005             41
               ... 
Jun 28, 2009      1
Jul 29, 2019      1
Jun 22, 1985      1
Jul 11, 2009      1
Nov 10, 2018      1
Name: Start_Aired, Length: 4348, dtype: int64

In [ ]:
int(anime["Start_Aired"].iloc[2].split()[-1])

2009

In [ ]:
errores = pd.DataFrame()
for pos in range(len(anime["Start_Aired"])):
    try:
        if type(anime["Start_Aired"].iloc[pos]) == str:
            # anime["Start_Aired"][pos]
            anime["Start_Aired"].iloc[pos]=int(anime["Start_Aired"].iloc[pos].split()[-1])
    except:
        x=anime["Start_Aired"].iloc[pos]
        errores.append([pos,x])



In [ ]:
errores

""


In [ ]:
#19 siempre es 2019. Anime comienza en 1917.

anime["Start_Aired"].value_counts()

2016      341
2014      334
2017      317
2012      307
2015      303
         ... 
Mar-57      1
Oct-95      1
Apr-80      1
Apr-95      1
Jun-19      1
Name: Start_Aired, Length: 129, dtype: int64

In [ ]:
errores = pd.DataFrame()
for pos in range(len(anime["Start_Aired"])):
    try:
        if type(anime["Start_Aired"].iloc[pos]) == str:
            aniored=int(anime["Start_Aired"].iloc[pos].split("-")[-1])
            if aniored<=24:
                anio=2000+aniored
            else:
                anio=1900+aniored          
            anime["Start_Aired"].iloc[pos]=anio
    except:
        x=anime["Start_Aired"].iloc[pos]
        errores.append([pos,x])

In [ ]:
anime["Start_Aired"].value_counts()

2016    341
2014    334
2017    318
2012    307
2018    305
       ... 
1933      1
1940      1
1943      1
1951      1
1952      1
Name: Start_Aired, Length: 92, dtype: int64

In [ ]:
#Eliminamos las filas donde Start_Aired == Unknown
# anime = anime.drop(index=anime.loc[anime['Start_Aired'] == 'Unknown'].index)
#No funciona, nos elimina también los valores donde aparecen solo años. ¿Mejor cambiar todos los Unknown por un outlier, en plan 1900?
print(type(anime["Start_Aired"][0]))


<class 'str'>


In [ ]:
anime["Start_Aired"].value_counts()
#¿No debería salir 126 de 1900?... No entiendo nada... Y si hago .info() esta columna ya no es int, sino object... pero lo cambié por un número...

Unknown         126
2020             56
2010             50
2011             43
2005             41
               ... 
Jun 28, 2009      1
Jul 29, 2019      1
Jun 22, 1985      1
Jul 11, 2009      1
Nov 10, 2018      1
Name: Start_Aired, Length: 4348, dtype: int64

In [ ]:
#Vamos a intentar mantener sólo el año de cada fila

# anime["Start_Aired"] =anime["Start_Aired"].replace('(\d{4})')
anime["Start_Aired"].value_counts

<bound method IndexOpsMixin.value_counts of 0         Apr 7, 2013
1         Oct 4, 2006
2         Apr 5, 2009
3         Oct 5, 2015
5         Apr 3, 2016
             ...     
21452            2013
21453            2014
21455    May 12, 2018
21456    Aug 22, 2020
21459    Nov 10, 2018
Name: Start_Aired, Length: 7919, dtype: object>

In [ ]:
anime["Start_Aired"] = anime["Start_Aired"].astype("int64")

ValueError: invalid literal for int() with base 10: 'Apr 7, 2013'

In [ ]:
# #Vamos a intentar mantener sólo el año de cada fila
# #Convertir columna a formato fecha
# anime["Start_Aired"]=pd.to_datetime(anime["Start_Aired"])
# #Extraer el año de cada fecha
# anime["Start_Aired"] =anime["Start_Aired"].dt.year
# anime["Start_Aired"]= anime["Start_Aired"].astype("int64")
# anime["Start_Aired"].value_counts

In [ ]:
anime.columns

Index(['Title', 'English', 'Type', 'Episodes', 'Status', 'Start_Aired',
       'Premiered', 'Licensors', 'Studios', 'Source', 'Genres', 'Themes',
       'Demographics', 'Duration_Minutes', 'Rating', 'Score', 'Scored_Users',
       'Ranked', 'Popularity', 'Members', 'Favorites'],
      dtype='object')

In [ ]:
anime["Premiered"].value_counts() #Se repite, borrar columna

Unknown        5524
Spring 2014      42
Spring 2017      42
Spring 2016      40
Spring 2011      38
               ... 
Summer 1977       1
Summer 1978       1
Winter 1993       1
Winter 1967       1
Summer 1976       1
Name: Premiered, Length: 214, dtype: int64

In [ ]:
anime["Studios"].value_counts()

Unknown                                      3345
Toei Animation                                512
Shanghai Animation Film Studio                308
TMS Entertainment                             215
OLM                                           178
                                             ... 
TMS Entertainment, Telecom Animation Film       1
Anpro                                           1
Studio Kikan                                    1
helo.inc                                        1
Original Force                                  1
Name: Studios, Length: 507, dtype: int64

In [ ]:
anime['Source'].value_counts()

Manga           2589
Original        2307
Unknown         1741
Game             309
Other            285
Picture book     132
4-koma manga     111
Novel            100
Book              78
Visual novel      73
Light novel       58
Music             54
Web manga         42
Mixed media       25
Card game         12
Radio              3
Name: Source, dtype: int64

In [ ]:
anime['Rating'].value_counts()

G - All Ages                      2981
PG-13 - Teens 13 or older         2167
PG - Children                     1974
R - 17+ (violence & profanity)     442
R+ - Mild Nudity                   277
None                                78
Name: Rating, dtype: int64

In [ ]:
# 'Popularity', 'Members', 'Favorites'
anime['Popularity'].value_counts()

21254    2
20063    2
19687    2
6649     2
19688    2
        ..
7435     1
7417     1
7414     1
7412     1
21490    1
Name: Popularity, Length: 7781, dtype: int64

In [ ]:
anime['Members'].value_counts()

29       69
22       61
31       54
32       51
30       49
         ..
32356     1
32319     1
32303     1
32298     1
10431     1
Name: Members, Length: 4248, dtype: int64

In [ ]:
anime['Favorites'].value_counts()

0       3763
1        666
2        312
3        176
4        157
        ... 
2545       1
3235       1
1457       1
862        1
854        1
Name: Favorites, Length: 979, dtype: int64

In [ ]:
anime.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7919 entries, 0 to 21459
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Title             7919 non-null   object 
 1   English           7919 non-null   object 
 2   Type              7919 non-null   object 
 3   Episodes          7919 non-null   float64
 4   Status            7919 non-null   object 
 5   Start_Aired       7919 non-null   object 
 6   Premiered         7919 non-null   object 
 7   Studios           7919 non-null   object 
 8   Source            7919 non-null   object 
 9   Genres            7919 non-null   object 
 10  Themes            7919 non-null   object 
 11  Demographics      7919 non-null   object 
 12  Duration_Minutes  7919 non-null   float64
 13  Rating            7919 non-null   object 
 14  Score             7919 non-null   float64
 15  Scored_Users      7919 non-null   float64
 16  Ranked            7919 non-null   float64